In [ ]:
import networkx as nx
import numpy as np
import pandas as pd
from math import sqrt
import csv
import sys

In [ ]:
from sklearn.cluster import DBSCAN
from scipy.sparse import *

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
from pyBallMapper_Bokeh import graph_GUI

In [ ]:
from matplotlib.colors import ListedColormap
from matplotlib import cm

In [ ]:
from bokeh.plotting import figure, show

In [ ]:
# to deal with large csv
maxInt = sys.maxsize
decrement = True

while decrement:
    # decrease the maxInt value by factor 10
    # as long as the OverflowError occurs.
    decrement = False
    try:
        csv.field_size_limit(maxInt)
    except OverflowError:
        maxInt = int(maxInt/10)
        decrement = True

In [ ]:
# ## Read the graphs

# Each graph must be rapresented by an adjecency list (space separated)
# We assume nodes are numbered from 1 to N
#
# The list of points covereb by each node is a file with N lines, each line contains the points id (space separated)

def read_graph_from_list(GRAPH_ADJ_PATH, GRAPH_POINTS_PATH):
    # read graph adjecency list
    # G_dummy is needed because I want the nodes to be ordered
    # ASSUME NODES ARE NUMBERED FROM 1 TO N
    G_dummy = nx.read_adjlist(GRAPH_ADJ_PATH, nodetype = int,
                              delimiter=' ')

    # read list of points covered by each node
    # ASSUME NODES ARE NUMBERED FROM 1 TO N
    csv_file = open(GRAPH_POINTS_PATH)
    reader = csv.reader(csv_file)

    points_covered = {}
    MAX_NODE_SIZE = 0
    for i, line_list in enumerate(reader):
        points_covered[i+1] = [int(node) for node in line_list[0].split(' ')]
        if len(points_covered[i+1]) > MAX_NODE_SIZE:
            MAX_NODE_SIZE = len(points_covered[i+1])

    # add the nodes that are not in the edgelist
    G = nx.Graph()
    G.add_nodes_from( range(1, len(points_covered) + 1) )
    G.add_edges_from(G_dummy.edges)

    MIN_SCALE = 7
    MAX_SCALE = 20

    for node in G.nodes:
        G.nodes[node]['points covered R'] = points_covered[node]
        G.nodes[node]['points covered'] = (np.array(points_covered[node])-1).tolist()
        G.nodes[node]['size'] = len(G.nodes[node]['points covered'])
        # rescale the size for display
        G.nodes[node]['size rescaled'] = MAX_SCALE*G.nodes[node]['size']/MAX_NODE_SIZE + MIN_SCALE

    return G

In [ ]:
# mapper on BM using DBscan as clustering algo
# it uses scipy csr sparse matrix to speed up computations
# inputs:
#     BM_J        ball mapper graph
#     dense_K_df  pandas dataframe where to pull back elements in the BM
#     EPS         radius for the DBscan algo
#     MIN_SAMPLES min number of elements in a cluster that make it a cluster and not noise
# https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html

def mapper_on_BM(BM_J, dense_K_df, EPS, MIN_SAMPLES=1):
    new_graph = nx.Graph()

    # creates a sparse CSR matrix
    sparse_K = csr_matrix(dense_K_df.values)

    for node in tqdm(BM_J.nodes):
        X = sparse_K[BM_J.nodes[node]['points covered'], :]

        db = DBSCAN(eps=EPS, min_samples=MIN_SAMPLES).fit(X)
        # create a set of unique labels
        labels = set(db.labels_) - {-1} # outliers are not clusters

        #print('\n **********')
        #print(node, X.shape[0], labels)

        # for each cluster
        # add a new vertex to the new graph
        for cluster in labels:
            # print the number of points in the cluster
            #print('\t', cluster, (db.labels_ == cluster).sum())
            # retrives the indeces of the points covered by the cluster
            points_covered_by_cluster = np.array(BM_J.nodes[node]['points covered'])[np.where(db.labels_
                                                                                     == cluster)].tolist()
            # creates a node
            new_graph.add_node(str(node)+'_'+str(cluster),
                               points_covered=points_covered_by_cluster)

        for neigh in [v for v in nx.neighbors(BM_J, node) if v > node]:
            neigh_X = sparse_K[BM_J.nodes[neigh]['points covered'], :]

            neigh_db = DBSCAN(eps=EPS, min_samples=MIN_SAMPLES).fit(neigh_X)
            neigh_labels = set(neigh_db.labels_) - {-1} # outliers are not clusters

            # add edges between clusters that belongs to neigh in the original graph
            # if they share at least one element
            for cluster in labels:
                for neigh_cluster in neigh_labels:
                    points_covered_by_cluster = np.array(BM_J.nodes[node]['points covered'])[np.where(db.labels_
                                                                                             == cluster)].tolist()
                    points_covered_by_neigh=np.array(BM_J.nodes[neigh]['points covered'])[np.where(neigh_db.labels_
                                                                                          == neigh_cluster)].tolist()
                    if len( set(points_covered_by_cluster)&set(points_covered_by_neigh) ) != 0:
                        new_graph.add_edge(str(node)+'_'+str(cluster), str(neigh)+'_'+str(neigh_cluster) )


    return new_graph

In [ ]:
# we will save the mapper_on_BM to disk as pickle files 
# this way we can plot them in a second moment

In [ ]:
# Read the mapper_on_BM graph from pickle

def read_graph_from_pickle(GRAPH_PATH,
                           values_df,
                           my_palette):
    # read graph 
    G = nx.read_gpickle(GRAPH_PATH)
    
    MIN_SCALE = 7
    MAX_SCALE = 20

    MAX_NODE_SIZE = 0
    for node in G.nodes:
        if len(G.nodes[node]['points_covered']) > MAX_NODE_SIZE:
            MAX_NODE_SIZE = len(G.nodes[node]['points_covered'])

    for node in G.nodes:
        G.nodes[node]['size'] = len(G.nodes[node]['points_covered'])
        # rescale the size for display
        G.nodes[node]['size rescaled'] = MAX_SCALE*G.nodes[node]['size']/MAX_NODE_SIZE + MIN_SCALE

        G.nodes[node]['color'] = my_palette(0)

        for name, avg in values_df.loc[G.nodes[node]['points_covered']].mean().iteritems():
            G.nodes[node][name] = avg

    return G

# EXAMPLE
## Jones upto13n TO Khovanov upto13n

In [ ]:
print('up to 13n SYMM 20 JONES')

BM_J_13n = read_graph_from_list('output/jones_fromK_upto_13n_MIRRORS/SYM_20_edges',
                               'output/jones_fromK_upto_13n_MIRRORS/SYM_20_points_covered_by_landmarks')

K_13n_df = pd.read_csv('output/jones_fromK_upto_13n_MIRRORS/Khovanov_upto_13n_MIRRORS_coeff.csv', 
                       sep=' ')

print('data loaded')
print('computing mapper on BM')
pullback_13n_15 = mapper_on_BM(BM_J_13n, K_13n_df,
                               EPS=15, MIN_SAMPLES=1)

nx.write_gpickle(pullback_13n_15, 'pullback_13n_15_no_thrs_SYMM.gpickle')

In [ ]:
from bokeh.models import FixedTicker, LinearColorMapper, LogColorMapper, ColorBar, BasicTicker, LogTicker
from matplotlib.colors import to_hex

In [ ]:
GRAPH1_PATH = 'pullback_13n_15_no_thrs_SYMM.gpickle'

# table with the coloring functions
coloring_df = pd.read_csv('output/jones_fromK_upto_13n_MIRRORS/Khovanov_upto_13n_MIRRORS_colors.csv',
                          sep=' ')
coloring_df.index = range(len(coloring_df))

coloring_df['signature mod4'] = coloring_df.signature % 4

###########
# GRAPH 1 #
###########

#Here we adopt standard colour palette
my_red_palette = cm.get_cmap(name='jet')

# read graph
# ASSUME NODES ARE NUMBERED FROM 1 TO N
G1 = read_graph_from_pickle(GRAPH1_PATH, coloring_df, my_red_palette)

for node in G1.nodes:
    G1.nodes[node]['points covered'] = G1.nodes[node]['points_covered']
print('loaded graph with {} nodes and {} edges'.format(len(G1.nodes), len(G1.edges)))

# create a GUI with input our BM graph, 
# a dataframe with coloring functions (one value per point in the pointcloud)
# and a color palette
# in this case we use the pointcloud as coloring function
my_fancy_gui = graph_GUI(G1, my_red_palette, coloring_df[['signature']])
my_fancy_gui.color_by_variable('signature')

In [ ]:
# discrete colorbar
num_ticks = 9
low = -9
high = 9
color_mapper = LinearColorMapper(palette=[to_hex(my_red_palette(color_id)) 
                                          for color_id in np.linspace(0, 1, num_ticks)], 
                                 low=low, high=high)

ticks = [i for i in range(-12, 13, 2)]
color_ticks = FixedTicker(ticks=ticks)

color_bar = ColorBar(color_mapper=color_mapper, 
                     major_label_text_font_size='14pt',
                     label_standoff=12,
                     ticker=color_ticks,
                    )

In [ ]:
my_fancy_gui.plot.add_layout(color_bar, 'right')

In [ ]:
# creates an html file with the graph 
# and opens it in another tab
show(my_fancy_gui.plot)